In [ ]:
import tensorflow as tf
print(tf.__version__)

2.4.0


In [ ]:
!pip install kaggle

In [ ]:
!pwd

/content


In [ ]:
# getting kaggle.json file from google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!mkdir -p ~/.kaggle
!cp '/content/drive/My Drive/kaggle/kaggle.json' ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c digit-recognizer

  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 31.6MB/s]
 82% 5.00M/6.09M [00:00<00:00, 42.3MB/s]
100% 6.09M/6.09M [00:00<00:00, 38.9MB/s]
 55% 5.00M/9.16M [00:00<00:00, 22.6MB/s]
100% 9.16M/9.16M [00:00<00:00, 36.2MB/s]


In [ ]:
# Extracting the dataset
# !unzip 'train.csv.zip'
# !unzip 'test.csv.zip'

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [ ]:
# Importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf

from subprocess import check_output
print(check_output(["ls","../content"]).decode("utf8"))

drive
sample_data
sample_submission.csv
test.csv
test.csv.zip
train.csv
train.csv.zip



In [ ]:
# reading train and test data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.head(10)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train.shape

(42000, 785)

In [ ]:
test.shape

(28000, 784)

In [ ]:
train_y = train['label'].astype('int32')
train_x = train.drop(['label'],axis =1).astype('int32')
test_x = test.astype('float32')
train_x.shape,train_y.shape,test_x.shape

((42000, 784), (42000,), (28000, 784))

In [ ]:
train_x = train_x.values.reshape(-1,28,28,1)
train_x = train_x/255.0
test_x = test_x.values.reshape(-1,28,28,1)
test_x = test_x/255.0
train_x.shape, test_x.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [ ]:
# One Hot Encoding
train_y = tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(42000, 10)

In [ ]:
# printing original labels of top 5 rows
print(train['label'].head())

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64


In [ ]:
# One hot encoding of the same labels
print(train_y[0:5,:])

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
  tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),

  tf.keras.layers.Conv2D(64,(3,3), activation ='relu', padding='Same'),
  tf.keras.layers.Conv2D(64,(3,3), activation ='relu', padding='Same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides= (2,2)),
  tf.keras.layers.Dropout(0.25),

  tf.keras.layers.Conv2D(64,(3,3), activation ='relu', padding='Same'),
  tf.keras.layers.Conv2D(64,(3,3), activation ='relu', padding='Same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides= (2,2)),
  tf.keras.layers.Dropout(0.25),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation = 'relu'),
  tf.keras.layers.Dense(256, activation = 'relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(10, activation = 'softmax'),                                    
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 6, 64)         

In [ ]:
# Defining call back function to stop training when reached a particular accuracy
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nReached 999% accuarcy so halting training!")
      self.model.stop_training  =True
callbacks = myCallback()

In [ ]:
#Compiling and model trainign with batch size= 50, epochs =20, and  optimizer = Adam
Optimizer = tf.keras.optimizers.Adam(
            learning_rate = 0.0005,
            beta_1=0.9,
            beta_2=0.999,
            epsilon = 1e-07,
            name = 'Adam'
)
model.compile(optimizer=Optimizer, loss = 'categorical_crossentropy',metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=50, epochs=20, callbacks=[callbacks])

Epoch 1/20
840/840 [==============================] - 133s 157ms/step - loss: 0.7884 - accuracy: 0.7215
Epoch 2/20
840/840 [==============================] - 131s 156ms/step - loss: 0.0835 - accuracy: 0.9738
Epoch 3/20
840/840 [==============================] - 132s 157ms/step - loss: 0.0598 - accuracy: 0.9818
Epoch 4/20
840/840 [==============================] - 132s 158ms/step - loss: 0.0510 - accuracy: 0.9844
Epoch 5/20
840/840 [==============================] - 131s 156ms/step - loss: 0.0402 - accuracy: 0.9881
Epoch 6/20
840/840 [==============================] - 131s 156ms/step - loss: 0.0348 - accuracy: 0.9900
Epoch 7/20
840/840 [==============================] - 130s 155ms/step - loss: 0.0302 - accuracy: 0.9906
Epoch 8/20
840/840 [==============================] - 131s 156ms/step - loss: 0.0248 - accuracy: 0.9926
Epoch 9/20
840/840 [==============================] - 130s 155ms/step - loss: 0.0249 - accuracy: 0.9924
Epoch 10/20
840/840 [==============================] - 130s 155m

In [ ]:
results = model.predict(test_x)

# index with max probablity
results = np.argmax(results, axis=1)
results = pd.Series(results, name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"), results], axis=1)

submission.to_csv("submission.csv", index=False)

In [ ]:
 # Submitting on Kaggle
 !kaggle competitions submit -c digit-recognizer -f submission.csv -m "Message"

100% 208k/208k [00:02<00:00, 93.7kB/s]
Successfully submitted to Digit Recognizer